<a href="https://colab.research.google.com/github/semkud/NLP/blob/main/HW3/HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
!pip install pymorphy2
import pymorphy2
import string
import re
import pandas as pd
from math import log
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

morph = pymorphy2.MorphAnalyzer()
vectorizer1 = CountVectorizer(binary = True)
vectorizertf = TfidfVectorizer(use_idf=False)

In [ ]:
!pip install nltk
!pip install spacy
!pip  install pyLDAvis
!python3 -m spacy download en
import nltk; nltk.download('stopwords')

In [4]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [5]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [ ]:
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
print(df.target_names.unique())
print(df['content'][0])

In [7]:

# Convert to list
data = df.content.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[0])

#tikenize

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[0][2])

('From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: '
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 'this funky looking car, please e-mail. Thanks, - IL ---- brought to you by '
 'your neighborhood Lerxst ---- ')
my


In [8]:

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])


/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting_host', 'rac_wam_umd_edu', 'organization', 'university', 'of', 'maryland_college_park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


In [9]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [10]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['where', 'thing', 'car', 'nntp_poste', 'host', 'park', 'line', 'wonder', 'could', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'bricklin', 'door', 'really', 'small', 'addition', 'separate', 'rest', 'body', 'know', 'tellme', 'model', 'name', 'engine', 'year', 'production', 'car', 'make', 'history', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst']]


In [11]:
print(data_lemmatized[0:10])

[['where', 'thing', 'car', 'nntp_poste', 'host', 'park', 'line', 'wonder', 'could', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'bricklin', 'door', 'really', 'small', 'addition', 'separate', 'rest', 'body', 'know', 'tellme', 'model', 'name', 'engine', 'year', 'production', 'car', 'make', 'history', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst'], ['poll', 'final', 'call', 'summary', 'final', 'call', 'clock', 'report', 'keyword', 'acceleration', 'line', 'host', 'fair', 'number', 'brave', 'soul', 'upgrade', 'clock', 'oscillator', 'share', 'experience', 'poll', 'send', 'brief', 'message', 'detailing', 'experience', 'procedure', 'top', 'speed', 'attain', 'cpu', 'rate', 'speed', 'add', 'card', 'hour', 'usage', 'day', 'functionality', 'floppy', 'especially', 'request', 'summarize', 'next', 'day', 'add', 'network', 'knowledge', 'base', 'do', 'clock', 'upgrade', 'answer', 'poll', 'thank'], ['engineering', 'compu

In [12]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 5), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1)]]


In [13]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[0:2]]
pprint(id2word[20])

'look'


Нахождение оптимального количества групп:

In [ ]:
prps = []
for a in range (1, 30, 1):
  lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=a, 
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
  print('\nPerplexity: ', lda_model.log_perplexity(corpus))
  doc_lda = lda_model[corpus]
  prps.append(lda_model.log_perplexity(corpus))
optimal_number =  prps.index(min(prps)) + 1

Я не запускаю в финальной версии эту ячейку, она очень долго работает. Когда запускал в тестовом режиме, получалось, что перплексити очень мало отличалось, буквально на десятые доли, что ставит вопрос, а то ли мы вообще считаем, что вот у нас для 2 групп и для 22 примерно один и тот же результат по этому показателю. Однако в задании надо было написать код, исходя из того, что чем ниже перплексити, тем лучше, и вот он написан. В дальнейшем будем считать, что оптимальное количество тем - 20, как и было изначально в датасете.

In [14]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [15]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.029*"physical" + 0.023*"week" + 0.021*"cause" + 0.017*"wire" + 0.016*"ca" '
  '+ 0.013*"sometimes" + 0.012*"appreciate" + 0.011*"pain" + 0.011*"advice" + '
  '0.011*"direct"'),
 (1,
  '0.027*"group" + 0.023*"gun" + 0.021*"report" + 0.015*"issue" + '
  '0.014*"receive" + 0.013*"year" + 0.013*"internet" + 0.013*"member" + '
  '0.013*"service" + 0.013*"send"'),
 (2,
  '0.041*"faith" + 0.022*"church" + 0.017*"sin" + 0.016*"christian" + '
  '0.015*"man" + 0.011*"love" + 0.011*"diagnosis" + 0.011*"christianity" + '
  '0.010*"tumor" + 0.010*"body"'),
 (3,
  '0.082*"car" + 0.026*"slave" + 0.022*"engine" + 0.022*"brain" + '
  '0.017*"model" + 0.015*"pay" + 0.014*"room" + 0.013*"typical" + 0.012*"door" '
  '+ 0.012*"bill"'),
 (4,
  '0.024*"source" + 0.023*"may" + 0.023*"provide" + 0.017*"use" + '
  '0.014*"information" + 0.013*"reference" + 0.012*"contain" + '
  '0.011*"research" + 0.011*"server" + 0.010*"point"'),
 (5,
  '0.020*"kill" + 0.020*"people" + 0.019*"child" + 0.017*"death" +

Часть кода, которая выдаваемые топики режет на ключевые слова и их веса

In [18]:
topics = []
for topic in lda_model.print_topics():
  nums = []
  words = []
  string = topic[1]
  nws = re.split(' \+ ', string)
  for nw in nws:
    num, word = re.split('\*', nw)
    nums.append(num)
    words.append(word[1:-1])
  topics.append([words, nums])


print(topics[0])
print(topics[2])

[['physical', 'week', 'cause', 'wire', 'ca', 'sometimes', 'appreciate', 'pain', 'advice', 'direct'], ['0.029', '0.023', '0.021', '0.017', '0.016', '0.013', '0.012', '0.011', '0.011', '0.011']]
[['faith', 'church', 'sin', 'christian', 'man', 'love', 'diagnosis', 'christianity', 'tumor', 'body'], ['0.041', '0.022', '0.017', '0.016', '0.015', '0.011', '0.011', '0.011', '0.010', '0.010']]


Для каждого текста в датасете считаем, какие слова, в него входящие входят в какой топик, прибавляем вес, выбираем максимальное значение

In [19]:
for_df = []
for text in data_lemmatized:
  topic_counter = [0 for topic in topics]
  for word in text:
    for topic in topics:
      if word in topic[0]:
        topic_counter[topics.index(topic)] += float(topic[1][topic[0].index(word)])
  broad_theme = topic_counter.index(max(topic_counter))
  for_df.append([text, broad_theme])

In [20]:
datafr = pd.DataFrame(for_df)
datafr #Получили датафрейм из текстов (лемматизированных слов) и номера топика

,0,1
0,"[where, thing, car, nntp_poste, host, park, li...",3
1,"[poll, final, call, summary, final, call, cloc...",8
2,"[engineering, computer, network, distribution_...",9
3,"[division, line, host, write, write, article, ...",8
4,"[question, distribution, article, write, clear...",15
...,...,...
11309,"[scan, city, reply, line, consultation, cheap,...",9
11310,"[screen, medford, old, problem, screen, blank,...",0
11311,"[este, mount, mail, group, line, instal, try, ...",8
11312,"[line, article, write, boy, embarasse, trivial...",8


Собираем тексты одного топика в одну длинную строку.

In [44]:
longtexts = []
for a in range(0, len(topics), 1):
  mylist = datafr[datafr[1] == a][0].values.tolist()
  print('Theme - ', a, 'количество текстов - ', len(mylist))
  longstr = ''
  for text in mylist:
    for word in text:
      longstr += ' '+word
  longtexts.append(longstr)
  print('длина текстов на тему', len(longstr))

Theme -  0 количество текстов -  22
длина текстов на тему 12676
Theme -  1 количество текстов -  56
длина текстов на тему 95107
Theme -  2 количество текстов -  50
длина текстов на тему 54818
Theme -  3 количество текстов -  276
длина текстов на тему 208994
Theme -  4 количество текстов -  45
длина текстов на тему 84447
Theme -  5 количество текстов -  42
длина текстов на тему 76160
Theme -  6 количество текстов -  33
длина текстов на тему 48187
Theme -  7 количество текстов -  167
длина текстов на тему 88416
Theme -  8 количество текстов -  7029
длина текстов на тему 2671987
Theme -  9 количество текстов -  1397
длина текстов на тему 1630509
Theme -  10 количество текстов -  101
длина текстов на тему 81856
Theme -  11 количество текстов -  110
длина текстов на тему 163603
Theme -  12 количество текстов -  3
длина текстов на тему 1766
Theme -  13 количество текстов -  14
длина текстов на тему 36446
Theme -  14 количество текстов -  512
длина текстов на тему 382061
Theme -  15 количеств

Делаем ТФИДФ

In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [47]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(longtexts)
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
df_tf_idf = pd.DataFrame(denselist, columns=feature_names)
df_tf_idf

,aa_vj,aaa,aaah,aaef,aalternate,aam,aamaze,aamir_qazi,aangegeven,aantal,aaplay,aargh,aario,aaron,aaron_ray,aarskog,aaske,aazo,ab,abad,abandon,abandonde,abandonment,abate,abatement,abbasid,abberant,abberation,abbot,abbotts,abbreviate,abbreviation,abcs,abdi,abdication,abdoman,abdomen,abdominal,abduct,abducted,...,zombie,zombo,zonal,zond,zone,zoning,zoo,zoological,zoologist,zoom,zorastrian,zorg,zorn,zoroaster,zoroastrian,zoroastrianism,zortech,zqr,zqy,zset,zsoft,zterm,ztimer,zubov,zue,zug,zulu,zuma,zumdahl,zumder,zurbrin,zurlo,zurvanism,zv,zvfd,zw,zyxel,zzd,zzzs,zzzzzzt
0,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.00000
1,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.004987,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.003623,0.000000,0.000000,...,0.000000,0.000000,0.004987,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.00000
2,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00636,0.00636,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.00000
3,0.000000,0.001165,0.000000,0.00000,0.000000,0.00000,0.000000,0.002544,0.00000,0.00000,0.00000,0.000000,0.000000,0.001165,0.000000,0.000000,0.00000,0.000000,0.011651,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.001768,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.00000
4,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.032291,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000,0.0000

Вынимаем слова с самым большим тфидф

In [49]:
import numpy as np
feature_names = np.array(vectorizer.get_feature_names())
def get_top_tf_idf_words(tfidf_vector, feature_names, top_n):
    sorted_nzs = np.argsort(tfidf_vector.data)[:-(top_n+1):-1]
    return feature_names[tfidf_vector.indices[sorted_nzs]]
wordlist = []
for i in range(0, 19, 1):
  article_vector = vectors[i, :]
  words = get_top_tf_idf_words(article_vector, feature_names, 5)
  print(words)
  wordlist.append([i, words])

['ca' 'plane' 'wire' 'reincarnation' 'pain']
['gun' 'report' 'homicide' 'group' 'year']
['faith' 'church' 'sin' 'believe' 'say']
['car' 'would' 'engine' 'write' 'line']
['may' 'internet' 'privacy' 'information' 'user']
['kill' 'people' 'armenian' 'child' 'village']
['power' 'ground' 'system' 'water' 'state']
['sale' 'price' 'include' 'sell' 'line']
['line' 'write' 'would' 'article' 'host']
['would' 'go' 'be' 'say' 'know']
['db_db' 'bit' 'color' 'scsi' 'use']
['evidence' 'atheist' 'exist' 'religion' 'believe']
['bike' 'steal' 'harley' 'ride' 'part']
['ax' 'max' 'di_di' 'bhjn' 'wm_wm']
['game' 'team' 'player' 'year' 'play']
['people' 'would' 'say' 'write' 'make']
['space' 'launch' 'satellite' 'mission' 'orbit']
['israeli' 'monitor' 'gazan' 'font' 'black']
['file' 'program' 'window' 'image' 'available']


Запишем в итоговый датафрейм:


In [51]:
datafr = pd.DataFrame(for_df, columns=['text', 'broad_theme'])
datafr2 = pd.DataFrame(wordlist, columns=['broad_theme', 'top_5_words'])
datafr3 = pd.merge(datafr, datafr2, on = 'broad_theme')
datafr3

,text,broad_theme,top_5_words
0,"[where, thing, car, nntp_poste, host, park, li...",3,"[car, would, engine, write, line]"
1,"[insurance, rate, performance, car, line, late...",3,"[car, would, engine, write, line]"
2,"[car, saftey, important, line, host, write, sa...",3,"[car, would, engine, write, line]"
3,"[steal, line, write, front, panel, car, stereo...",3,"[car, would, engine, write, line]"
4,"[need, information, partially, responsible, ca...",3,"[car, would, engine, write, line]"
...,...,...,...
11085,"[zionism, racism, line, jewish, lead, israeli,...",17,"[israeli, monitor, gazan, font, black]"
11086,"[character, miss, line, host, word, show, peri...",17,"[israeli, monitor, gazan, font, black]"
11087,"[steal, line, article, write, article, write, ...",12,"[bike, steal, harley, ride, part]"
11088,"[richard_sturge, help, bike, short, reply, ric...",12,"[bike, steal, harley, ride, part]"
